# **Алгоритм для получения данных о конкретном криптокошельке**

## **<span style="color:blue"> Описание**
**Данный алгоритм на основе ранеее полученного в параметры функции адреса кашелька bitcoin отправляет запрос на получение данных о нём.
После чего эти данные специальным образом обрабатываются и передаются в виде словаря Python.**

In [7]:
%pip install datetime
%pip install urllib3

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/43.1 kB ? eta -:--:--
     -------------------------------------- - 41.0/43.1 kB 1.9 MB/s eta 0:00:01
     ---------------------------------------- 43.1/43.1 kB 1.1 MB/s eta 0:00:00
  Using cached setuptools-69.2.0-py3-none-any.whl.metadata (6.3 kB)
   ---------------------------------------- 0.0/52.6 kB ? eta -:--:--
   ---------------------------------------- 52.6/52.6 kB 2.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/204.5 kB ? eta -:--:--
   ------------------------ --------------- 122.9/204.5 kB 2.4 MB/s eta 0:00:01
   ---------------------------------------- 204.5/204.5 kB 3.1 MB/s eta 0:00:00
Using cached setuptools-69.2.0-py3-none-any.whl (821 kB)
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel t

## **<span style="color:blue"> Необходимые библиотеки**
**datetime** — модуль позволяющий управлять датами и временем, представляя их в таком виде, в котором пользователи смогут их понимать.  
**urllib3** — мощный HTTP-клиент на Python c простым для понимания и продуманным кодом. Она поддерживает безопасность потоков, пул соединений.

In [10]:
import json, time, datetime
import urllib.request

## **<span style="color:blue"> Функции**

### Функция <span style="color:red">convertTime  
Функция **convertTime** возвращает указанную метку времени в привычной для нас метрике.

In [ ]:
def convertTime(time):
    return datetime.datetime.strptime(datetime.datetime.fromtimestamp(int(time)).strftime('%Y-%m-%d %H:%M:%S'), '%Y-%m-%d %H:%M:%S')

### Функция <span style="color:red">getWalletData  
Функция **getWalletData** делает запрос на получение данных о кошельке, после чего сохраняет полученные данные в переменной data в формате json. Дата содержит множество параметров, например, баланс кошелька, количество транзакций, время транзакций и т.д. Эти данные мы используем для генерации словаря с такими пармаетрами как <span style="color:green">balance</span> - баланс, <span style="color:green">transaction</span> - количество транзакций, <span style="color:green">received</span> - сколько всего было получено, <span style="color:green">sent</span> - сколько всего было отправлено, <span style="color:green">midtime</span> - среднее время между операциями, <span style="color:green">class</span> - опасна ли транзакция( 1 если опасна и 0 если безопасна).

In [12]:
def getWalletData(address):
    try:
        with urllib.request.urlopen('https://blockchain.info/rawaddr/' + address) as url:
            data = json.loads(url.read())

        if len(data['txs'])-1 != 0:
            midtime = abs(sum((convertTime(data['txs'][num]['time']) - convertTime(data['txs'][num+1]['time'])).total_seconds()
                              for num in range(len(data['txs'])-1))/(len(data['txs'])-1))
        else:
            midtime = 0
        return {'balance' : data['final_balance'], 'transaction' : data['n_tx'], 'received' : data['total_received'], 
                'sent' : data['total_sent'], 'midtime' : int(midtime), 'class' : 0}
    except urllib.error.HTTPError as err:
        if err.code == 429:
            print('wait')
            time.sleep(10)
            return getWalletData(address)
        else:
            raise